## **Initialization**

In [0]:
import pyspark.sql.functions as F
from pyspark.sql.types import StringType
from pyspark.sql.functions import col, trim

## **Reading From Bronze Layer**

In [0]:
df = spark.table("workspace.bronze.erp_loc_a101")

## **Data Transformation**

### 1-Trimming String Type Data

In [0]:
for field in df.schema.fields:
    if isinstance(field.dataType, StringType):
        df = df.withColumn(field.name, trim(col(field.name)))

### 2-Customer ID Cleanup

In [0]:
df = (
    df
    .withColumn(
         "CID",
          F.expr("""
                    REPLACE(CID, '-', '')
          """)
     )
)

### 3-Country Data Normalization

In [0]:
df = (
    df
    .withColumn(
        "CNTRY", 
        F.expr(""" 
                    CASE 
                        WHEN UPPER(CNTRY) == 'DE' THEN 'Germany'
                        WHEN UPPER(CNTRY) == 'US' THEN 'United States'
                        WHEN CNTRY == '' OR CNTRY IS NULL THEN 'Unknown'
                        ELSE CNTRY
                    END 
        """)
    )
)



### 4-Renaming Columns

In [0]:
# Creating new column names in a dictionary
RENAME_MAP = {

    "CID": "customer_number",
    "CNTRY": "country"
}

# Looping for all columns names and rename them
for old_name, new_name in RENAME_MAP.items():
    df = df.withColumnRenamed(old_name, new_name)

## **Write Into Silver Layer**

In [0]:
df.write.mode("overwrite").format("delta").saveAsTable("workspace.silver.erp_customer_location")

## **Sanity Check Of Data Frame**

In [0]:
df.display()